In [1]:
addprocs()

@everywhere qs = 1.0 + 2.0.^(-5:2)
times = Array{Float64}(length(qs),4)
means = Array{Float64}(length(qs),4)

@everywhere begin
  using StochasticDiffEq, DiffEqBase, DiffEqProblemLibrary,
        Plots, LaTeXStrings, ParallelDataTransfer, DiffEqMonteCarlo
  srand(99 + myid())
  full_prob = oval2ModelExample(largeFluctuations=true,useBigs=false)
  prob = deepcopy(full_prob)
  prob.tspan=(0.0,1.0)
  println("Solve once to compile.")
  sol = solve(prob,EM(),dt=1/2^(18))
  Int(sol.u[end][1]!=NaN)
  println("Compilation complete.")
  num_runs = 10000

  probs = Vector{SDEProblem}(3)
  p1 = Vector{Any}(3)
  p2 = Vector{Any}(3)
  p3 = Vector{Any}(3)
  ## Problem 1
  probs[1] = prob_sde_linear
  ## Problem 2
  probs[2] = prob_sde_wave
  ## Problem 3
  probs[3] = prob_sde_additive
end
println("Setup Complete")

## Timing Runs

@everywhere function runAdaptive(i,k)
  sol = solve(prob,SRIW1(),dt=1/2^(8),abstol=2.0^(-15),reltol=2.0^(-10),maxIters=Int(1e12),qmax=qs[k])
  Int(any(isnan,sol[end]) || sol.t[end] != 1)
end

#Compile
monte_prob = MonteCarloProblem(probs[1])
test_mc = solve(monte_prob,SRIW1(),dt=1/2^(4),adaptive=true,num_monte=1000,abstol=2.0^(-1),reltol=0)
calculate_monte_errors(test_mc);

Solve once to compile.
Compilation complete.
	From worker 11:	Solve once to compile.
	From worker 11:	Compilation complete.
	From worker 14:	Solve once to compile.
	From worker 14:	Compilation complete.
	From worker 6:	Solve once to compile.
	From worker 6:	Compilation complete.
	From worker 5:	Solve once to compile.
	From worker 12:	Solve once to compile.
	From worker 12:	Compilation complete.
	From worker 10:	Solve once to compile.
	From worker 10:	Compilation complete.
	From worker 3:	Solve once to compile.
	From worker 3:	Compilation complete.
	From worker 16:	Solve once to compile.
	From worker 16:	Compilation complete.
	From worker 2:	Solve once to compile.
	From worker 13:	Solve once to compile.
	From worker 13:	Compilation complete.
	From worker 7:	Solve once to compile.
	From worker 4:	Solve once to compile.
	From worker 9:	Solve once to compile.
	From worker 9:	Compilation complete.
	From worker 15:	Solve once to compile.
	From worker 15:	Compilation complete.
	From worker 17

## qmax test on Oval2 Model

In [2]:
for k in eachindex(qs)
  ParallelDataTransfer.sendto(workers(), k=k)
  @everywhere srand(99 + myid())
  adaptiveTime = @elapsed numFails = sum(pmap((i)->runAdaptive(i,k),1:num_runs))
  println("k was $k. The number of Adaptive Fails is $numFails. Elapsed time was $adaptiveTime")
  times[k,4] = adaptiveTime
end

k was 1. The number of Adaptive Fails is 0. Elapsed time was 36.254151618
k was 2. The number of Adaptive Fails is 0. Elapsed time was 28.5158889
k was 3. The number of Adaptive Fails is 0. Elapsed time was 26.332680421
k was 4. The number of Adaptive Fails is 0. Elapsed time was 26.457054035
k was 5. The number of Adaptive Fails is 0. Elapsed time was 27.698225713
k was 6. The number of Adaptive Fails is 0. Elapsed time was 28.750677513
k was 7. The number of Adaptive Fails is 0. Elapsed time was 29.873483577
k was 8. The number of Adaptive Fails is 0. Elapsed time was 29.451160315


## qmax test on other problems

In [5]:
for k in eachindex(probs)
  println("Problem $k")
  ## Setup
  prob = probs[k]
  ParallelDataTransfer.sendto(workers(), prob=prob)

  for i in eachindex(qs)
    ParallelDataTransfer.sendto(workers(), i=i)
    msim = solve(monte_prob,dt=1/2^(4),SRIW1(),adaptive=true,num_monte=num_runs,abstol=2.0^(-13),reltol=0,qmax=qs[i])
    test_msim = calculate_monte_errors(msim)
    times[i,k] = test_msim.elapsedTime
    means[i,k] = test_msim.error_means[:final]
    println("for k=$k and i=$i, we get that the error was $(means[i,k]) and it took $(times[i,k]) seconds")
  end
end

Problem 1
for k=1 and i=1, we get that the error was 4.315400143790057e-6 and it took 8.140950299 seconds
for k=1 and i=2, we get that the error was 4.89756583367057e-6 and it took 5.807222325 seconds
for k=1 and i=3, we get that the error was 8.506156247652843e-6 and it took 6.227678904 seconds
for k=1 and i=4, we get that the error was 4.354179640359905e-6 and it took 5.896393505 seconds
for k=1 and i=5, we get that the error was 6.1666476750771586e-6 and it took 5.842827081 seconds
for k=1 and i=6, we get that the error was 5.445323348038243e-6 and it took 5.757022574 seconds
for k=1 and i=7, we get that the error was 5.405930291172606e-6 and it took 5.944903097 seconds
for k=1 and i=8, we get that the error was 8.698820539062227e-6 and it took 7.373844157 seconds
Problem 2
for k=2 and i=1, we get that the error was 4.307280325997364e-6 and it took 6.017691704 seconds
for k=2 and i=2, we get that the error was 4.83150423203309e-6 and it took 6.011026717 seconds
for k=2 and i=3, we g